# Environment (Linux)

In [1]:
# conda create -n llm python=3.10
# conda activate llm 
! pip install chardet
! pip install tqdm
! pip install pandas
! pip install -i https://pypi.tuna.tsinghua.edu.cn/simple pandas numpy ipywidgets tqdm
! pip install -i https://pypi.tuna.tsinghua.edu.cn/simple torch==2.1.2  transformers==4.38.2 datasets tiktoken wandb==0.11 openpyxl
! pip install -i https://pypi.tuna.tsinghua.edu.cn/simple peft==0.8.0 accelerate bitsandbytes safetensors jsonlines
! pip install -i https://pypi.tuna.tsinghua.edu.cn/simple vllm==0.3.1
! pip install -i https://pypi.tuna.tsinghua.edu.cn/simple trl==0.7
! pip install -i https://pypi.tuna.tsinghua.edu.cn/simple tensorboardX tensorboard
! pip install -i https://pypi.tuna.tsinghua.edu.cn/simple textdistance nltk matplotlib seaborn seqeval
! pip install -i https://pypi.tuna.tsinghua.edu.cn/simple modelscope
# May require:
# pip install --upgrade numpy

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


# Import Packages

In [2]:
import os
import codecs
import chardet
import time
from tkinter import *
from tkinter.filedialog import askdirectory
from tkinter import messagebox
import tqdm
import json
import re
import glob
import csv
import ast
import pandas as pd
import shutil
import torch
from transformers import  AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging
from peft import LoraConfig, PeftModel
from vllm import LLM, SamplingParams
import math
import numpy as np

# 1. Convert to UTF-8 encoding

In [3]:
def encoding_2_utf8(filename,out_enc="UTF-8"):
    """
    Converts the encoding of a given file to UTF-8. 
    If the file is already in UTF-8, no conversion is performed.

    Parameters:
    filename (str): The path of the file to be converted.
    out_enc (str): The target encoding, default is UTF-8.
    """
    try:
        content = open(filename, 'rb').read()
        # Detect the original encoding of the file
        source_encoding = chardet.detect(content)
        # If the file is not in UTF-8, perform the conversion
        if(source_encoding['encoding'] != "utf-8"):
            if(source_encoding['encoding'] == 'GB2312'):
                content = content.decode("GBK",errors='ignore')
                content = content.encode(out_enc)
                codecs.open(filename, 'wb').write(content)
            elif(source_encoding['encoding'] == None):
                pass
            else:
                content = content.decode(source_encoding['encoding'],errors='ignore').encode(out_enc)
                codecs.open(filename, 'wb').write(content)
        else:
            pass

    except IOError as err:
        print("I/O error:{0}".format(err))
    # Function to explore all .txt files in a specified directory
def process_txt_files_in_directory(dir):
    for root, dirs, files in os.walk(dir):
        for file in tqdm.tqdm(files):
            if os.path.splitext(file)[1] == '.txt':
                path = os.path.join(root, file)
                # Call encoding_2_utf8 function to convert the file encoding from GBK to UTF-8
                encoding_2_utf8(path)

### Convert all .txt files in a specified directory to UTF-8 encoding

In [4]:
#Replace with your folder path
director_path = "./demo_article"
#Convert to UTF-8 encoding
process_txt_files_in_directory(director_path)

100%|██████████| 4/4 [00:00<00:00, 22.38it/s]


# 2. Use regular expressions extract NMR paragraph


In [6]:
# Use regular expressions to extract NMR paragraphs
def extract_C_with_compound_new(files_contain_H, output_path_H):
    # Traverse the files in the folder
    for file in tqdm.tqdm(files_contain_H):
        # Initialize a dictionary to save the content of the article
        line_dict = {
            'file_name': '', 'Paragraph': []
        }
        # Open the file
        with open(file, 'r', encoding='utf-8') as H:
            # Read lines
            lines = H.readlines()
            # Record the total number of lines to prevent index overflow when concatenating lines
            len_all = len(lines)
            # Traverse all lines to get paragraphs that may contain NMR data
            for i, line in enumerate(lines):
                # Remove whitespace characters from the line
                line = line.strip()
                # Temporarily store the line for concatenation
                line_add1 = line
                line_line = line
                '''
                In order to obtain paragraphs containing NMR data 
                and eliminate common writing differences such as 13C-NMR, 
                13CNMR, and 13C NMR, we use the regular expression 
                13C.{0,3} NMR for paragraph matching, which means matching 13C 
                followed by 0 to 3 arbitrary characters and then NMR.
                '''
                # Use the regex pattern_C = re.compile(r'13C.{0,3}NMR', re.I) to check if the line may contain NMR data, 
                # and this line is also the starting line for the carbon spectrum NMR data description in the paper
                # (13C.{0,3}NMR is a common starting mark for carbon spectrum NMR data description in the text).
                res = pattern_C.findall(line)
                # If the line may contain NMR data, further concatenate the previous and next lines 
                # to ensure the extracted NMR paragraph contains complete NMR data; otherwise, skip it.
                if res:
                    # Use the find_compound_function to check if the line contains IUPAC name information of the compound
                    comp_res = find_compound_function(line)
                    # If the line contains IUPAC name information, perform: 1. concatenate one line upwards. 
                    # 2. concatenate multiple lines downwards; otherwise, skip it.
                    if comp_res:
                        # 1. Concatenate one line upwards
                        # If the previous line is empty, attempt to concatenate the second line upwards.
                        # (If lines[i - 1] is an empty line, then not lines[i - 1].strip() will be True; otherwise, it will be False)
                        if not lines[i - 1].strip() and not pattern_C.findall(lines[i - 2]):
                            line_add1 = lines[i - 2].strip() + ' ' + line_add1
                        # If the previous line is not empty, attempt to concatenate the previous line.
                        elif not pattern_C.findall(lines[i - 1]):
                            line_add1 = lines[i - 1].strip() + ' ' + line_add1
                        # 2. Concatenate multiple lines downwards
                        for z in range(1, 4):
                            # Prevent index overflow
                            if i + z <= len_all - 1:
                                # The lines to concatenate downwards must not contain IUPAC name information of the compound, 
                                # and must not contain the starting information of other compounds' carbon spectrum data; otherwise, stop concatenating,
                                # (to avoid multiple compounds' NMR data in one NMR paragraph)
                                if find_compound_function(lines[i + z]) == False and not pattern_C.findall(lines[i + z]):
                                    line_add1 += ' ' + lines[i + z].strip()
                                else:
                                    break    
                        # 3. Temporarily store the concatenated NMR paragraph
                        line_dict['Paragraph'].append(line_add1)
                    # If the line does not contain IUPAC name information of the compound, perform: 
                    # 1. attempt to concatenate one line upwards. 
                    # 2. if the concatenated previous line still does not contain IUPAC name information, continue to concatenate upwards. 
                    # 3. concatenate multiple lines downwards; otherwise, skip it.
                    else:
                        # Flag to indicate whether the upward concatenation has been executed
                        linde_add1_lable = 1
                        # 1. Attempt to concatenate one line upwards
                        # If the previous line is empty, attempt to concatenate the second line upwards.
                        if not lines[i - 1].strip() and not pattern_C.findall(lines[i - 2]):
                            line_line = lines[i - 2].strip() + ' ' + line_line
                        # If the previous line is not empty, attempt to concatenate the previous line.
                        elif not pattern_C.findall(lines[i - 1]):
                            line_line = lines[i - 1].strip() + ' ' + line_line
                        # If no upward concatenation has been executed
                        else:
                            linde_add1_lable = 0

                        # 2. If the concatenated previous line still does not contain IUPAC name information, continue to concatenate upwards.
                        for j in range(2, 4):
                            # If upward concatenation has not been executed for one line, the previous line may contain other compounds' carbon spectrum NMR data, so break the loop
                            if linde_add1_lable == 0:
                                break
                            # Check if the concatenated previous line contains IUPAC name information of the compound
                            #(The previous line's emptiness is not considered here, which does not affect the continuation of upward concatenation,
                            # because even if two empty lines are concatenated, it will not affect the completeness of the paragraph.)
                            comp_res0 = find_compound_function(lines[i - 1])
                            # If the concatenated previous line does not contain IUPAC name information of the compound, 
                            # and does not contain other compounds' carbon spectrum data description, then continue attempting to concatenate upwards.
                            if comp_res0 == False and not pattern_C.findall(lines[i - 1]):
                                line_last = lines[i - j].strip()
                                # Check if the line to be concatenated contains IUPAC name information
                                comp_res1 = find_compound_function(line_last)
                                # If there is no IUPAC name information of the compound, 
                                # and no description of other compounds' carbon spectrum data, concatenate the line and continue to attempt to concatenate upwards; otherwise, break the loop.
                                if comp_res1 == False and not pattern_C.findall(line_last):
                                    line_line = line_last + ' ' + line_line
                                else:
                                    line_line = line_last + ' ' + line_line
                                    break
                            else:
                                break
                        # 3. Concatenate multiple lines downwards
                        for z in range(1, 4):
                            # Prevent index overflow
                            if i + z <= len_all - 1:
                                # The lines to concatenate downwards must not contain IUPAC name information of the compound, 
                                # and must not contain the starting information of other compounds' carbon spectrum data; otherwise, stop concatenating,
                                # (to avoid multiple compounds' NMR data in one NMR paragraph)
                                if find_compound_function(lines[i + z]) == False and not pattern_C.findall(lines[i + z]):
                                    line_line += ' ' + lines[i + z].strip()
                                else:
                                    break
                        # 4. Temporarily store the concatenated NMR paragraph
                        line_dict['Paragraph'].append(line_line)
        # Generate the corresponding JSON file name from the TXT file name
        output_file_name = os.path.basename(file)[:-4] + '.json'
        # Create a folder to store the JSON files containing NMR paragraphs
        if not os.path.exists(output_path_H):
            os.makedirs(output_path_H)
        # Write the concatenated NMR paragraphs into the JSON file
        if line_dict['Paragraph'] != '':
            line_dict['file_name'] = (os.path.basename(file)[:-4])
            with open(os.path.join(output_path_H, output_file_name), 'w',) as w:
                w.write(json.dumps(line_dict, indent=4))

# Method to check if the current line contains IUPAC name information of the compound
def find_compound_function(line):
    kword = re.split('[ ()\[\]{}/;,-]', line)
    for word in kword:
        # Ignore words that are too short
        if len(word) < 3:
            continue
        for compound in compounds:
            if len(compound) > len(word):
                break
            if compound in word:
                return True
    return False

# File containing 618 common compound IUPAC name groups
dict_path = r'618_common_word_groups_of_IUPAC_name.json'
Dictionary_compound = open(dict_path, 'r', encoding='utf-8')
dict_compound = json.loads(Dictionary_compound.read())['compounds']
compounds = dict_compound.keys()
# Sort the compound names from longest to shortest for easier matching
compounds = sorted(compounds, key=lambda x: len(x))
# Regular expression to determine if a line may contain NMR data; 13C.{0,3}NMR is a common starting mark for carbon spectrum NMR data description in the text
pattern_C = re.compile(r'13C.{0,3}NMR', re.I)


### Extract the paragraphs containing NMR data in the article

In [7]:
#Original txt file location
txt_original_path= rf'./demo_article'
text_files = glob.glob(txt_original_path + '/*.txt')
#Output the corresponding json file location
output_C = rf'./2_demo_article_paragraph_json'
extract_C_with_compound_new(text_files, output_C)


100%|██████████| 2/2 [00:00<00:00, 25.59it/s]


### Filter json files containing NMR data

In [8]:
def get_files_with_C(C_path,paths):
    C_files = glob.glob(C_path + '/*.json')
    C_with_compound_paths = paths
    if not os.path.exists(C_with_compound_paths):
        os.makedirs(C_with_compound_paths)
    for file in tqdm.tqdm(C_files):
        with open(file, 'r', encoding='utf-8') as H:
            data = json.loads(H.read())
            if data['Paragraph']:
                shutil.copy2(file, C_with_compound_paths)


if __name__ == '__main__':
    c_path = rf'./2_demo_article_paragraph_json'
    c_compound_path = rf'./3_demo_article_NMR_paragraph_json'
    get_files_with_C(c_path,c_compound_path)

100%|██████████| 2/2 [00:00<00:00, 529.42it/s]


### Save the json file containing NMR data into csv

In [9]:
H_files = rf'./3_demo_article_NMR_paragraph_json'

gpt_path = rf'./4_demo_article_NMR_paragraph_csv'
if not os.path.exists(gpt_path):
    os.makedirs(gpt_path)
txt_files_contain_C = sorted(glob.glob(H_files + '/*.json'))
df1 = {'file_name': [], 'Paragraph': []
        }
for file in txt_files_contain_C:

    with open(file, 'r', encoding='utf-8') as H:
        data_list = json.load(H)

        sheet_pasragraph = data_list['Paragraph']   

        for line in sheet_pasragraph:
                output_file_name = os.path.basename(file)[:-5]
                df1['file_name'].append(output_file_name)
                df1['Paragraph'].append(line)
# Convert the list of JSON data into a DataFrame
df = pd.DataFrame(df1)
#df.to_csv(os.path.join(gpt_path,f'PMC00{number}.csv'),index=False)
df.to_csv(os.path.join(gpt_path,f'demo_article_NMR_paragraph.csv'),index=False)
print(df)


      file_name                                          Paragraph
0   PMC10339406  https://creativecommons.org/licenses/by/4.0/ P...
1   PMC10339406  Scheme 1 Synthesis Pathway for Obtained Compou...
2   PMC10339406  3 Conclusions Among the antifungal drug groups...
3   PMC10339406  4.1.5.1 2-((5-(2,3-Diethylquinoxalin-6-yl)-4-e...
4   PMC10339406  4.1.5.2 2-((5-(2,3-Diethylquinoxalin-6-yl)-4-e...
5   PMC10339406  4.1.5.3 4-(2-((5-(2,3-Diethylquinoxalin-6-yl)-...
6   PMC10339406  4.1.5.4 2-((5-(2,3-Diethylquinoxalin-6-yl)-4-e...
7   PMC10339406  4.1.5.5 2-((5-(2,3-Diethylquinoxalin-6-yl)-4-e...
8   PMC10339406  4.1.5.6 1-(4-Chlorophenyl)-2-((5-(2,3-diethylq...
9   PMC10339406  4.1.5.7 1-(4-Bromophenyl)-2-((5-(2,3-diethylqu...
10  PMC10339406  4.1.5.8 2-((5-(2,3-Diethylquinoxalin-6-yl)-4-e...
11  PMC10339406  4.1.5.9 2-((5-(2,3-Diethylquinoxalin-6-yl)-4-e...
12  PMC10339406  4.1.5.10 1-(2,4-Dichlorophenyl)-2-((5-(2,3-die...
13  PMC10339406  4.1.5.11 1-(3,4-Dichlorophenyl)-2-((5-(2,3-di

# 3. Use LLMs extract NMR data 


In [10]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

device = "cuda:0" if torch.cuda.is_available() else "cpu"

dir = f"./5_demo_article_NMR_data"
if not os.path.exists(dir):
    os.makedirs(dir)
# Replaced with NMRExtractor model weight path
# The model weights of NMRExtractor can be downloaded from https://huggingface.co/sweetssweets/NMRExtractor. 
model_path = f"sweetssweets/NMRExtractor"
device_map = {"": 0}                                                                                          # Load the entire model on the GPU 0
sampling_params = SamplingParams(temperature=0, top_p=1,max_tokens = 4096, stop = ['!!!'])

# Create an LLM.
# llm = LLM(model="pretrained_models/llama/llama-2-7b-chat-hf")
llm = LLM(model=model_path)


prom = '''Extract text containing 1H NMR and 13C NMR data, remove interference information such as reactants, raw materials, solvents and other non-final product names based on text semantics, and then extract the name, code or number of the final product. Please delete the IUPAC name Alias, numbers and ordinal numbers before and after fields, such as '2.1.3.', '(HL4)', '(9)', '(4d)'. NMR text should contain complete information, such as instrument power and solvent information, For example, "13C NMR text": "13C NMR (400 MHz, acetone-d6) 174.0 (C), 157.7 (C). Then split the NMR text. The content in NMR conditions is NMR instrument power and solvent information, such as "13C NMR conditions": "400MHz, acetone-d6". The content in the 13C NMR data removes information such as the position and shape of the peak,  such as "13C NMR data": "131.4-128.0, 157.7". The content in the 1H NMR data should include information such as the position and shape of the peak, such as "1H NMR data": "12.57 (s, 1H), 7.97-7.95 (d, J = 8.25 Hz, 2H)". Please keep the duplicate values of the original data and do not modify the number of decimal places. All responses must originate from information extracted from the given text, ensuring that the extracted content has not been modified or fragmented, and that capitalization and punctuation are exactly the same as the given text. Must end with {"IUPAC":"text","1H NMR text":"text","1H NMR conditions":"text","1H NMR data":"text","13C NMR text":"text","13C NMR conditions":"text","13C NMR data":"text"} format reply.'''


test_df = pd.read_csv(os.path.join(gpt_path,f'demo_article_NMR_paragraph.csv'), encoding='utf-8')


# Remove leading and trailing spaces from the Paragraph column
test_df['Paragraph'] = test_df['Paragraph'].str.strip()
prompts = []

for i in range(len(test_df)):
    
    prompt = f"{prom} {test_df['Paragraph'][i]}"
    prompts.append(f"<s>[INST] {prompt} [/INST]")
print(len(prompts))
for data in prompts[:5]:
    print(data)

import math
import re
# Regular expression pattern to extract text
pattern = r'format reply\.(.*?)\[\/INST\]'

# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(prompts, sampling_params)


predictions = []
predictions_prob = []

# Create an empty collection to record the request_id that has been processed
processed_ids = set()
for output in outputs:
    request_id = output.request_id
    # Check if the current request_id is already in the collection
    if request_id not in processed_ids:
        # Add request_id to the collection to indicate that it has been processed
        processed_ids.add(request_id)
    else:
        # If it already exists, skip it
        continue
    prompt = output.prompt
    generated_text = output.outputs[0].text 

    # Use regular expressions to find matches
    match = re.search(pattern, prompt)
    
    predictions.append(generated_text)
    predictions_prob.append(math.exp(output.outputs[0].cumulative_logprob))
pred_df = pd.DataFrame()
pred_df['File Name'] = test_df['file_name']
pred_df['Paragraph'] = test_df['Paragraph']
pred_df['Generated Text Prob'] = predictions_prob
pred_df['Generated Text'] = predictions


#pred_df.to_csv(f"{dir}/5_demo_article_NMR_data.csv", index = None)


INFO 10-25 16:43:07 llm_engine.py:79] Initializing an LLM engine with config: model='./checkpoint-1600', tokenizer='./checkpoint-1600', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)
INFO 10-25 16:43:16 llm_engine.py:337] # GPU blocks: 27422, # CPU blocks: 2048
INFO 10-25 16:43:21 model_runner.py:666] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-25 16:43:21 model_runner.py:670] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_

Processed prompts: 100%|██████████| 43/43 [00:30<00:00,  1.39it/s]


### Processing Data

In [11]:
def safe_json_loads(val):
    
    try:
        if val is np.nan:
            return {}
        else:
            return json.loads(str(val).replace("'", "\""))  # Convert all values to strings and attempt to parse
        
    except (ValueError, TypeError, json.JSONDecodeError):
        # Define regex pattern
        pattern = r'"(IUPAC|1H NMR text|1H NMR conditions|1H NMR data|13C NMR text|13C NMR conditions|13C NMR data)":"(.*?)"'
        # print('val', val)
        # Use regex to find all matching key-value pairs
        matches = re.findall(pattern, val)

        # Store results in a dictionary
        result = {key: value for key, value in matches}

        # Define all keys to look for
        keys = ["IUPAC", "1H NMR text", "1H NMR conditions", "1H NMR data", "13C NMR text", "13C NMR conditions", "13C NMR data"]

        # For each key, if it does not exist in the result dictionary, set its value to 'N/A'
        for key in keys:
            result[key] = result.get(key, 'N/A')
        return result

# Convert the string dictionary in column B to an actual dictionary
pred_df['Generated Text'] = pred_df['Generated Text'].apply(safe_json_loads)

# Use json_normalize to expand the dictionary into multiple columns and merge with the original DataFrame
pred_df = pd.concat([pred_df.drop('Generated Text', axis=1), pd.json_normalize(pred_df['Generated Text'])], axis=1)

pred_df.replace('nan', np.NaN, inplace=True)
pred_df = pred_df.fillna('N/A')

pred_df.fillna('N/A', inplace=True)

pred_df.rename(columns={'File Name': 'PMC accession number (PMCID)'}, inplace=True)
pred_df.rename(columns={'Generated Text Prob': 'Confidence in data given by large models'}, inplace=True)
pred_df.rename(columns={'1H NMR data': '1H NMR chemical shift'}, inplace=True)
pred_df.rename(columns={'13C NMR data': '13C NMR chemical shift'}, inplace=True)
pred_df.rename(columns={'IUPAC': 'IUPAC name'}, inplace=True)

pred_df = pred_df[pred_df['IUPAC name'] != 'N/A']
pred_df = pred_df[pred_df['13C NMR chemical shift'] != 'N/A']
pred_df.to_csv(f"{dir}/Extracted_NMRdata.csv", index=None)
pred_df


,PMC accession number (PMCID),Paragraph,Confidence in data given by large models,IUPAC name,1H NMR text,1H NMR conditions,1H NMR chemical shift,13C NMR text,13C NMR conditions,13C NMR chemical shift
3,PMC10339406,"4.1.5.1 2-((5-(2,3-Diethylquinoxalin-6-yl)-4-e...",0.933959,"2-((5-(2,3-Diethylquinoxalin-6-yl)-4-ethyl-4H-...","1H-NMR (300 MHz, DMSO-d6): δ = 1.27 (3H, t, J ...","300 MHz, DMSO-d6","1.27 (3H, t, J = 7.2 Hz, -CH3), 1.33–1.38 (6H,...","13C-NMR (75 MHz, DMSO-d6): δ =11.96, 12.01, 15...","75 MHz, DMSO-d6","11.96, 12.01, 15.48, 21.70, 27.86, 40.34, 41.2..."
4,PMC10339406,"4.1.5.2 2-((5-(2,3-Diethylquinoxalin-6-yl)-4-e...",0.923399,"2-((5-(2,3-Diethylquinoxalin-6-yl)-4-ethyl-4H-...","1H-NMR (300 MHz, DMSO-d6): δ = 1.27 (3H, t, J ...","300 MHz, DMSO-d6","1.27 (3H, t, J = 7.2 Hz, -CH3), 1.33–1.39 (6H,...","13C-NMR (75 MHz, DMSO-d6): δ =11.95, 12.01, 27...","75 MHz, DMSO-d6","11.95, 12.01, 27.85, 40.35, 56.14, 114.52, 127..."
5,PMC10339406,"4.1.5.3 4-(2-((5-(2,3-Diethylquinoxalin-6-yl)-...",0.921345,"4-(2-((5-(2,3-Diethylquinoxalin-6-yl)-4-ethyl-...","1H-NMR (300 MHz, DMSO-d6): δ = 1.27 (3H, t, J ...","300 MHz, DMSO-d6","1.27 (3H, t, J = 7.2 Hz, -CH3), 1.33–1.38 (6H,...","13C-NMR (75 MHz, DMSO-d6): δ =11.97, 12.03, 15...","75 MHz, DMSO-d6","11.97, 12.03, 15.45, 27.86, 40.40, 41.14, 116...."
6,PMC10339406,"4.1.5.4 2-((5-(2,3-Diethylquinoxalin-6-yl)-4-e...",0.899203,"2-((5-(2,3-Diethylquinoxalin-6-yl)-4-ethyl-4H-...","1H-NMR (300 MHz, DMSO-d6): δ = 1.28 (3H, t, J ...","300 MHz, DMSO-d6","1.28 (3H, t, J = 7.2 Hz, -CH3), 1.33–1.38 (6H,...","13C-NMR (75 MHz, DMSO-d6): δ =11.96, 12.02, 15...","75 MHz, DMSO-d6","11.96, 12.02, 15.46, 27.85, 41.32, 124.39, 127..."
7,PMC10339406,"4.1.5.5 2-((5-(2,3-Diethylquinoxalin-6-yl)-4-e...",0.931877,"2-((5-(2,3-Diethylquinoxalin-6-yl)-4-ethyl-4H-...","1H-NMR (300 MHz, DMSO-d6): δ = 1.28 (3H, t, J ...","300 MHz, DMSO-d6","1.28 (3H, t, J = 7.2 Hz, -CH3), 1.33–1.38 (6H,...","13C-NMR (75 MHz, DMSO-d6): δ =11.96, 12.02, 15...","75 MHz, DMSO-d6","11.96, 12.02, 15.47, 27.85, 40.37, 41.11, 116...."
8,PMC10339406,"4.1.5.6 1-(4-Chlorophenyl)-2-((5-(2,3-diethylq...",0.911375,"1-(4-Chlorophenyl)-2-((5-(2,3-diethylquinoxali...","1H-NMR (300 MHz, DMSO-d6): δ = 1.27 (3H, t, J ...","300 MHz, DMSO-d6","1.27 (3H, t, J = 7.2 Hz, -CH3), 1.33–1.38 (6H,...","13C-NMR (75 MHz, DMSO-d6): δ =11.97, 12.02, 15...","75 MHz, DMSO-d6","11.97, 12.02, 15.47, 27.87, 40.24, 41.09, 128...."
9,PMC10339406,"4.1.5.7 1-(4-Bromophenyl)-2-((5-(2,3-diethylqu...",0.915568,"1-(4-Bromophenyl)-2-((5-(2,3-diethylquinoxalin...","1H-NMR (300 MHz, DMSO-d6): δ = 1.27 (3H, t, J ...","300 MHz, DMSO-d6","1.27 (3H, t, J = 7.2 Hz, -CH3), 1.33–1.38 (6H,...","13C-NMR (75 MHz, DMSO-d6): δ =11.08, 12.01, 15...","75 MHz, DMSO-d6","11.08, 12.01, 15.43, 27.91, 39.09, 40.28, 41.1..."
10,PMC10339406,"4.1.5.8 2-((5-(2,3-Diethylquinoxalin-6-yl)-4-e...",0.942558,"2-((5-(2,3-Diethylquinoxalin-6-yl)-4-ethyl-4H-...","1H-NMR (300 MHz, DMSO-d6): δ = 1.26 (3H, t, J ...","300 MHz, DMSO-d6","1.26 (3H, t, J = 7.2 Hz, -CH3), 1.33–1.38 (6H,...","13C-NMR (75 MHz, DMSO-d6): δ = 11.97, 12.02, 1...","75 MHz, DMSO-d6","11.97, 12.02, 15.44, 21.40, 27.85, 43.05, 126...."
11,PMC10339406,"4.1.5.9 2-((5-(2,3-Diethylquinoxalin-6-yl)-4-e...",0.918773,"2-((5-(2,3-Diethylquinoxalin-6-yl)-4-ethyl-4H-...","1H-NMR (300 MHz, DMSO-d6): δ = 1.27 (3H, t, J ...","300 MHz, DMSO-d6","1.27 (3H, t, J = 7.2 Hz, -CH3), 1.33–1.38 (6H,...","13C-NMR (75 MHz, DMSO-d6): δ =11.97, 12.03, 15...","75 MHz, DMSO-d6","11.97, 12.03, 15.43, 27.85, 40.35, 44.27, 105...."
12,PMC10339406,"4.1.5.10 1-(2,4-Dichlorophenyl)-2-((5-(2,3-die...",0.926405,"1-(2,4-Dichlorophenyl)-2-((5-(2,3-diethylquino...","1H-NMR (300 MHz, DMSO-d6): δ = 1.24 (3H, t, J ...","300 MHz, DMSO-d6","1.24 (3H, t, J = 7.3 Hz, -CH3), 1.31–1.37 (6H,...","13C-NMR (75 MHz, DMSO-d6): δ =11.97, 12.01, 15...","75 MHz, DMSO-d6","11.97, 12.01, 15.39, 27.85, 27.87, 40.34, 43.0..."
